<a href="https://colab.research.google.com/github/ChanceChallacombe/kaggle/blob/main/esm_score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! python --version

Python 3.8.10


In [2]:
! pip install torch torchvision tensorflow accelerate transformers datasets huggingface_hub evaluate datasets requests pandas sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.5/191.5 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 11.6 MB/s eta 0:00:00
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=02e82a9d06ac487988fa8788162556f09ec3da454dfc24

In [3]:
model_checkpoint = "facebook/esm2_t12_35M_UR50D"

In [4]:
import requests
from io import BytesIO
import pandas
import gc, torch
from accelerate import Accelerator
gc.collect()
torch.cuda.empty_cache()
accelerator = Accelerator()
device = accelerator.device
url="https://rest.uniprot.org/uniref/stream?compressed=true&fields=id%2Cname%2Ctypes%2Ccount%2Corganism%2Clength%2Cidentity%2Csequence&format=tsv&query=%28%28length%3A%5B4%20TO%20500%5D%29%20AND%20toxin%29%20AND%20%28identity%3A0.5%29"
uniprot_request = requests.get(url)
bio = BytesIO(uniprot_request.content)
df = pandas.read_csv(bio, compression='gzip', sep='\t')

In [5]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer
labels=2

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=labels)
#model.esm = model.esm.half()
#torch.backends.cuda.matmul.allow_tf32 = True
model = model.to(device)


Some weights of the model checkpoint at facebook/esm2_t12_35M_UR50D were not used when initializing EsmForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing EsmForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing EsmForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t12_35M_UR50D and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 

In [6]:
test_prot=tokenizer(df["Reference sequence"][0], return_tensors="pt")
test_prot.to(device)

{'input_ids': tensor([[ 0, 20,  4, 17,  5,  9, 15,  6, 17,  5, 15,  4,  8, 13,  4, 12,  5, 12,
          4,  5,  5,  4, 17, 20, 12, 17, 17, 18, 17,  7,  4, 12, 14,  9,  9,  8,
         18,  8, 14, 12, 16,  4,  4, 15,  4,  9,  6, 15,  7, 10, 10, 10,  5,  8,
          6, 17, 17, 15, 14, 14,  7,  4, 16,  4,  5, 15, 14, 15,  8, 13, 22,  2]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}

In [7]:
test_prot

{'input_ids': tensor([[ 0, 20,  4, 17,  5,  9, 15,  6, 17,  5, 15,  4,  8, 13,  4, 12,  5, 12,
          4,  5,  5,  4, 17, 20, 12, 17, 17, 18, 17,  7,  4, 12, 14,  9,  9,  8,
         18,  8, 14, 12, 16,  4,  4, 15,  4,  9,  6, 15,  7, 10, 10, 10,  5,  8,
          6, 17, 17, 15, 14, 14,  7,  4, 16,  4,  5, 15, 14, 15,  8, 13, 22,  2]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}

In [8]:
model(**test_prot)

SequenceClassifierOutput(loss=None, logits=tensor([[0.0372, 0.0239]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [10]:
import torch

with torch.no_grad():
    output = model(**test_prot)

In [11]:
output

SequenceClassifierOutput(loss=None, logits=tensor([[0.0372, 0.0239]], device='cuda:0'), hidden_states=None, attentions=None)